In [64]:
import os
import dill
import ipywidgets as widgets
from aepsych.plotting import plot_strat
from aepsych.server import AEPsychServer
from IPython.display import display
from ipywidgets import HTML, AppLayout, Box, Button, Label, Layout, Text
from IPython.core.display import HTML as html

connect_out = widgets.Output(layout=Layout(padding="2px", height="100px"))

database_path = None
server = AEPsychServer()
strat = None

#---------- Style ----------

display(html("""
<style>
.nav {
    box-shadow: 5px 5px 20px silver;
}

.jp-CodeCell {
    width:100% !important;
    background: whitesmoke; 
}
.jp-Notebook .jp-Cell {
    padding: 0 !important;
}

.jupyter-widgets.widget-tab {
    margin: 40px 30px;
}

.p-Collapse-header,
.jupyter-widgets.widget-tab > .p-TabBar .p-TabBar-tab.p-mod-current:before,
.jupyter-button ,
.widget-button, 
.jupyter-widgets.widget-tab > .widget-tab-contents {
    border-radius: 4px;
}


.jupyter-widgets.widget-tab > .p-TabBar .p-TabBar-tab{
border-top-right-radius:4px;
border-top-left-radius: 4px;
}
.p-Collapse-contents{
border-bottom-right-radius:4px;
border-bottom-left-radius: 4px;
}

.p-Collapse-open > .p-Collapse-header {
border-bottom-right-radius:0px;
border-bottom-left-radius: 0px;
}
body.jp-Notebook {
    margin: 0 !important;
    padding: 0 !important;
    height: 100vh !important;
    background: whitesmoke;
}
.widget-inline-hbox {
    margin: 0px;
}
.widget-output {
    padding: 10px !important;
    background: white;
    border-radius: 4px !important;
    box-shadow: inset 0 1px 1px rgb(0 0 0 / 5%);
    border: 1px solid #e3e3e3;
    border-radius: 4px;
    -webkit-box-shadow: inset 0 1px 1px rgb(0 0 0 / 5%);
    box-shadow: inset 0 1px 1px rgb(0 0 0 / 5%);
    margin: 5px;
    height: auto !important;
    width: auto !important;
   
}


.widget-text input[type="text"],
.widget-text input[type="number"], 
.widget-dropdown > select {
   border-radius: 4px;
   box-shadow: inset 0 1px 1px rgb(0 0 0 / 5%);
}

.widget-vbox {
    justify-content: center;
    width: 100%;
    width: -webkit-fill-available !important;
    display: flex;
    border-radius: 4px;
    background: white;
    height: fit-content;
    # box-shadow: inset 0 1px 1px rgb(0 0 0 / 5%);
}


}
[data-jp-theme-light='true'] .jp-RenderedImage img.jp-needs-light-background {
    width: 100%;
}

.jupyter-widgets.widget-tab {
    min-width: 30% !important;
}
html {
    background: #0091EA;
    height: 100%;
    margin: 0px; 
    padding: 0px; 

}

.widget-dropdown,
.widget-text{
    width: auto; 

}
.widget-label {
    display: flex;
    align-items: center;
}
.jp-RenderedText[data-mime-type='application/vnd.jupyter.stderr'] {
    background: white;
    padding-top: var(--jp-code-padding);
}
</style>"""))


header = HTML(
    """
<div class="nav" style=" background: #0091EA;margin: 0px;border:">
    
    <h1 style="
    text-align: center;
    font-family: helvetica;
    font-weight: 400;
    font-size: 20px;
    display: flex;
    padding: 15px 20px;
    margin: 0px;
    hieght: 100px; 
    jusify-content: flex-start;
    color: white">Interactive AEPsych</h1>
</div>
"""
)


input_style = {"description_width": "initial"}
input_layout= Layout(margin="5px 0")
btn_style = Layout(margin="10px",width="95%")

btn_box_layout = Layout(display="flex", justify_content="flex-end")
file_output = widgets.Output(
    layout={
            "overflow": "scroll", 
            "height": "50vh",
            "width": "100%",
            "padding": "5px"
           }
)


# ---------- Inputs -----------


def on_value_change(change):
    file_output.clear_output()
    with file_output:
        change["new"]
#         print(change['new'].keys())
#         print(change["new"])

        
def on_value_change_input(change):
    inputs_output.clear_output()
    with inputs_output:
        change["new"]


uploader = widgets.FileUpload(
    description="Select File",accept=".db", multiple=False, layout=btn_style)

yes_label = widgets.Dropdown(
    options=[("Yes Trial", "Detected Trial"), ("No Trial", "Undetected Trial")],
    value="Detected Trial",
    description="yes_label: ",
    style=input_style,
    layout=input_layout
)
no_label = widgets.Dropdown(
    options=[("No Trial", "Undetected Trial"), ("Yes Trial", "Detected Trial")],
    value="Undetected Trial",
    description="no_label: ",
    style=input_style,
    layout=input_layout
)

target_level = widgets.BoundedFloatText(
    value=0.75, min=0, max=1, 
    step=0.1, description="target_level:", disabled=False, 
    style=input_style, layout=input_layout
)

cred_level = widgets.BoundedFloatText(
    value=0.95, min=0, max=1, step=0.1, 
    description="cred_level:", disabled=False, 
    style=input_style, layout=input_layout
)

xlabel = Text(
    value="Angle (degrees)", placeholder="x axis label",
    description="xlabel: ", disabled=False, style=input_style,
    layout=input_layout
)

ylabel = Text(
    value="Detection Probability", placeholder="y axis label", 
    description="ylabel: ", disabled=False, style=input_style,
    layout=input_layout
)

# Observes input changes
uploader.observe(on_value_change, names="value")

inputs_output = widgets.Output(
    layout={"border": "1px solid black", "padding": "20px", "margin": "20px", "min-height ": "100px"}
)
yes_label.observe(on_value_change_input, names="value")
no_label.observe(on_value_change_input, names="value")
target_level.observe(on_value_change_input, names="value")
cred_level.observe(on_value_change_input, names="value")
xlabel.observe(on_value_change_input, names="value")
ylabel.observe(on_value_change_input, names="value")
# ------------ Buttons ------------

button_upload = widgets.Button(
    description="Upload", disabled=False, 
    button_style="success", tooltip="Click to Upload", 
    layout=btn_style
)
button_submit = widgets.Button(
    description="Submit", disabled=False, 
    button_style="success", tooltip="submit", 
    layout=btn_style
)
button_reset = widgets.Button(
    description="Reset", disabled=False, 
    button_style="info", tooltip="reset", layout=btn_style
)

button_server_reset = widgets.Button(
    description="Reset Session", disabled=False, 
    button_style="danger", tooltip="reset", layout=btn_style
)


# --------- Accordion -----------
accordion = widgets.Accordion(
    children=[
        widgets.VBox([no_label, yes_label], layout=Layout(padding="5px")),
        widgets.VBox(
            [
                target_level,
                cred_level,
            ],
            layout=Layout(padding="5px")
        ),
        widgets.VBox([xlabel, ylabel],layout=Layout(padding="5px")),
    ]
)
accordion.set_title(0, "Outcome Labels")
accordion.set_title(1, "Parameters")
accordion.set_title(2, "Axis Labels")

# ------------ Tabs -------------
tab = widgets.Tab(layout=Layout(height="100%", margin="0px 5px"))

children = [
    widgets.VBox(
        [
            Label("Resume Session:",layout=Layout(margin="10px")),
            uploader,
            button_upload,
            button_server_reset
        ],
         layout=Layout(margin="20px 10px", padding="5px")
    ),
    widgets.VBox(
        [
            accordion,
            widgets.Box([button_submit, button_reset], layout=btn_box_layout),
        ]
    ),
]
tab.children = children
tab.set_title(0, "Upload")
tab.set_title(1, "Plot")


#---------- Layout -----------

main_box_layout = widgets.Layout(
    height='100vh', 
    width='100%',
    grid_gap='5px',
    overflow='auto',
    align_items='stretch',
    margin='0px',
    border='0px',
    padding='0px',
    display='flex',
    flex_flow="column",
)


out = widgets.VBox(
    [Label("Output",
          layout=Layout(display="flex", justify_content="center", width="100%"))
     ,file_output],
    layout={"border": "1px solid #e3e3e3",
            "margin": "0px 5px", 
            "overflow": "scroll", 
            "width": "100%",
            "padding": "10px",
           }
)

server_output =  widgets.VBox([Label("Server Status",
                            layout=Layout(display="flex", justify_content="center", width="100%", height="auto",
                                         padding="2px",  overflow_y="auto")
                                    ), connect_out],
                             layout={
                                 "border": "1px solid #e3e3e3",
                                 "margin": "20px 3%",
                                 "padding": "5px",
                                 "width": "100%",
                                 "heigh": "auto",
                                 "max_height": "300px"
                             })

main_box = widgets.GridBox(
    children=[
        header,
        widgets.HBox(
        [out,tab], 
        layout=Layout(
        display='flex',
        flex_flow="row",
        margin="20px 2.55%"
        )),
        server_output
    ],
    layout=main_box_layout,
)

display(main_box)

# ------- functions ------------


def start_server():
    global strat
    global server
    with file_output:
        try:
            with connect_out: 
                server = AEPsychServer(database_path=database_path)
                strat = server.get_strat_from_replay()
#             print("Connected...")
            display_plot()
        except: 
            print("Unable to connect to server...")


def display_plot():
    global strat
    inputs_output.clear_output()
    if strat != None:
        plot_strat(
            strat,
            xlabel=xlabel.value,
            ylabel=ylabel.value,
            no_label=no_label.value,
            yes_label=yes_label.value,
            cred_level=cred_level.value,
            target_level=target_level.value,
        )
    else:
        print("Server has no experiment records!")


def upload():
    global server
    global strat
    global database_path
    connect_out.clear_output() 
    with file_output: 
        file_output.clear_output()
        if uploader.value == {}:
            print("No file uploaded")
        else:
            for file_name, file in uploader.value.items():
                current_path = os.getcwd()
                external_target_path = current_path + "/databases"
                completeName = None 
                
                if os.path.isdir(external_target_path):
                    completeName = os.path.join(external_target_path, file_name)
                else:
                    print("File upload unsuccessful")

                with open(completeName, "wb") as f:
                    f.write(file["content"])
                    if completeName != None:
                        database_path = completeName
                    start_server()
                    f.close()

def submit():
    file_output.clear_output()
    with file_output:
        display_plot()
        


def reset():
    with file_output:
        inputs_output.clear_output()
        xlabel.value = "Angle (degrees)"
        ylabel.value = "Detection Probability"
        yes_label.value = "Detected Trial"
        no_label.value = "Undetected Trial"
        cred_level.value = 0.75
        target_level.value = 0.95
        file_output.clear_output()


def server_reset():
    global database_path
    global server 
    global strat 
    global uploader 
    file_output.clear_output()
    connect_out.clear_output()
    uploader.value.clear()
    uploader._counter=0  
    server = AEPsychServer()
    database_path = None
    strat = None
   

# Onclick handler
def upload_clicked(b):
    upload()


def submit_clicked(b):
    submit()


def reset_clicked(b):
    reset()

    
def server_reset_clicked(b):
    server_reset()

    
button_server_reset.on_click(server_reset_clicked)
button_upload.on_click(upload_clicked)
button_submit.on_click(submit_clicked)
button_reset.on_click(reset_clicked)

2022-08-03 14:17:13,467 [INFO   ] Found DB at ./databases/default.db, appending!


GridBox(children=(HTML(value='\n<div class="nav" style=" background: #0091EA;margin: 0px;border:">\n    \n    …

2022-08-03 14:17:39,710 [INFO   ] Found DB at ./databases/default.db, appending!
